---
title: Downloading Climate Data from CHIRPS v3
short_title: CHIRPS v3 Climate Data
---

In this notebook we demonstrate how to fetch **CHIRPS v3 daily precipitation** for a DHIS2 organisation unit level, download only the needed spatial **bounding box**, and return an **xarray** object (ready for aggregation/analytics in a later step).

This notebook mirrors the ERA5-Land download flow, but uses CHIRPS v3 GeoTIFFs and raster window reads (no full global load).


--------------------------------------------
## Import libraries

We start by importing the necessary libraries:

In [2]:
import json
import geopandas as gpd

from dhis2eo.data.chc import chirps3

--------------------------------------------
## Load organisation units

Load organisation units for Sierra Leone. The resulting GeoDataFrame is used to derive a bounding box.

In [3]:
org_units = gpd.read_file('../../data/sierra-leone-districts.geojson')
org_units

,type,id,name,hasCoordinatesDown,hasCoordinatesUp,level,grandParentParentGraph,grandParentId,parentGraph,parentId,parentName,dimensions,weight,geometry
0,Polygon,O6uvpzGd5pu,Bo,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,Polygon,fdc6uOvgoji,Bombali,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,MultiPolygon,lc3eMKXaEfw,Bonthe,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,Polygon,jUb8gELQApl,Kailahun,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,MultiPolygon,PMa2VCrupOd,Kambia,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,Polygon,kJq2mPyFEHo,Kenema,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,Polygon,qhqAxPSTUXp,Koinadugu,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Polygon,Vth0fbpFcsO,Kono,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,MultiPolygon,jmIPBj66vD6,Moyamba,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,MultiPolygon,TEQlaapDQoK,Port Loko,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


--------------------------------------------
## Download CHIRPS v3 daily precipitation for the org unit bounding box

CHIRPS v3 data is provided at the daily level. We define the `start` and `end` parameters to only get precipitation data between the 1st and 7th July, and set the bounding box (`bbox`) to only get data for the spatial extent of the org units:

In [ ]:
# Choose a date range (inclusive)
start = "2025-07-01"
end = "2025-07-07"

ds = chirps3.daily.get(start=start, end=end, bbox=org_units.total_bounds)

dhis2eo.data.utils - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\dhis2eo_data_chc_chirps3_daily_get_b67190aa52.nc


If we inspect the contents of the downloaded dataset, we see that the main variable is precipitation (`precip`), and contains data for 7 days on the `time` dimension:

In [9]:
ds

<xarray.Dataset> Size: 107kB
Dimensions:  (time: 7, y: 62, x: 61)
Coordinates:
  * x        (x) float64 488B -13.27 -13.22 -13.17 ... -10.37 -10.32 -10.27
  * y        (y) float64 496B 9.975 9.925 9.875 9.825 ... 7.025 6.975 6.925
  * time     (time) datetime64[ns] 56B 2025-07-01 2025-07-02 ... 2025-07-07
Data variables:
    precip   (time, y, x) float32 106kB ...
Attributes:
    bbox:     [-13.3035   6.9176 -10.2658  10.0004]
    dataset:  CHIRPS v3 (daily)
    stage:    final
    flavor:   rnl